In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
// To make some of the examples work we will also need RDD
import org.apache.spark.rdd.RDD
import org.apache.log4j._
import org.apache.spark.sql._
import org.apache.spark.graphx.{Graph, VertexRDD}
import org.apache.spark.graphx.util.GraphGenerators

Intitializing Scala interpreter ...

Spark Web UI available at http://master.hadoop.lan:4042
SparkContext available as 'sc' (version = 3.0.0-preview, master = local[*], app id = local-1580541939438)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.log4j._
import org.apache.spark.sql._
import org.apache.spark.graphx.{Graph, VertexRDD}
import org.apache.spark.graphx.util.GraphGenerators


In [2]:
Logger.getLogger("org").setLevel(Level.ERROR)
val spark = SparkSession
    .builder
    .appName("graphx")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", "file:///tmp")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7bceb4e4


In [3]:
val sc=spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@58b22b09


In [4]:
val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Array((1L, ("jack", "owner")), (2L, ("george", "clerk")),
                       (3L, ("mary", "sales")), (4L, ("sherry", "owner wife"))))

users: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at <console>:42


In [5]:
users.foreach(println)

(3,(mary,sales))
(2,(george,clerk))
(4,(sherry,owner wife))
(1,(jack,owner))


In [6]:
val relationships: RDD[Edge[String]] =
  sc.parallelize(Array(Edge(1L, 2L, "boss"),    Edge(1L, 3L, "boss"),
                       Edge(2L, 3L, "coworker"), Edge(4L, 1L, "boss")))

relationships: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at parallelize at <console>:42


In [7]:
relationships.foreach(println)

Edge(1,3,boss)
Edge(4,1,boss)
Edge(1,2,boss)
Edge(2,3,coworker)


In [8]:
val defaultUser = ("", "Missing")

defaultUser: (String, String) = ("",Missing)


In [9]:
val graph = Graph(users, relationships, defaultUser)

graph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@4fd2552e


In [10]:
/*
The EdgeTriplet class extends the Edge class by adding the srcAttr and dstAttr members 
which contain the source and destination properties respectively. 
We can use the triplet view of a graph to render a collection of strings describing relationships between users.
*/

val facts: RDD[String] =
  graph.triplets.map(triplet =>
    triplet.srcAttr._1 + ", "+ triplet.srcAttr._2 + " is the " + triplet.attr + " of " + triplet.dstAttr._1+", "+triplet.dstAttr._2)
facts.collect.foreach(println(_))

jack, owner is the boss of george, clerk
jack, owner is the boss of mary, sales
george, clerk is the coworker of mary, sales
sherry, owner wife is the boss of jack, owner


facts: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[19] at map at <console>:48
